In [1]:
import sys
sys.path.append('../../build')
sys.path.append('../')
import IPSModule as ips
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import lattpy as lp

from utils import *

In [ ]:
gamma = 1
temperature = 0.3
epsilon = 1.0
sigma = 1.0
init_dis = 2 ** (1.0 / 6.0) * sigma
np.random.seed(42)

# set the radius of the cluster
rad_list = np.linspace(1.2, 4, 10)

# set the heat capacity and particle number list
particle_numbers = []
heat_capacities = []

# simulation parameters
num_step = 500000    
num_frames = 50      
draw_interval = num_step // num_frames  
equilibration_steps = int((num_step // 2) * 1.5)

for rad in rad_list:
    print(f"simulation rad = {rad}")
    num_p, pos = generate_circle_cluster(rad, 0, 0, init_dis, False)
    particle_numbers.append(num_p)
    
    p = ips.LangevinSystem(num_p, gamma, temperature)
    positions = pos
    for i in range(num_p):
        for d in range(2):
            p.get_positions()[d][i] = positions[i][d]
            p.get_velocities()[d][i] = 0.0

    pair_force_config = {
        "type": "LennardJones",
        "eps": epsilon,
        "sigma": sigma
    }

    confinement_config = {
        "type": "Radial",
        "rad": rad + 5.0
    }
    
    simulator = ips.IPS_Simulator_Langevin(p)
    simulator.init(pair_force_config, confinement_config)
    
    energy_arr = []
    
    def get_energy_callback(simulator, p, step):
        if step >= equilibration_steps:
            energy = calculate_total_energy(p.get_positions(), p.get_velocities(), epsilon=epsilon, sigma=sigma)
            energy_arr.append(energy)
    
    sim_visualizer = SimulationVisualizer(simulator=simulator, particle_system=p, 
                                          rad=rad, draw_interval=draw_interval, dt=0.001, show=False)
    sim_visualizer.add_callback(get_energy_callback)
    
    sim_visualizer.run_simulation(num_step)
    
    energy_arr = np.array(energy_arr)
    mean_E = np.mean(energy_arr)
    mean_E2 = np.mean(energy_arr**2)
    variance = mean_E2 - mean_E**2
    heat_capacity = variance / (temperature**2)
    heat_capacities.append(heat_capacity)


In [ ]:
plt.figure(figsize=(8,6))
plt.plot(particle_numbers, heat_capacities, 'o-', markersize=8)
plt.xlabel("number of particles")
plt.ylabel("heat capacity")
plt.title("Heat Capacity vs Number of Particles")
plt.grid(True)
plt.show()

In [ ]:
rad = 1.2

# simulation parameters
num_step = 100000
num_frames = 50      
draw_interval = num_step // num_frames  
equilibration_steps = num_step // 2 

print(f"simulation rad = {rad}")
num_p, pos = generate_circle_cluster(rad, 0, 0, init_dis, False)
particle_numbers.append(num_p)

p = ips.LangevinSystem(num_p, gamma, temperature)
positions = pos
for i in range(num_p):
    for d in range(2):
        p.get_positions()[d][i] = positions[i][d]
        p.get_velocities()[d][i] = 0.0

pair_force_config = {
    "type": "LennardJones",
    "eps": epsilon,
    "sigma": sigma
}

confinement_config = {
    "type": "Radial",
    "rad": rad + 5.0
}

simulator = ips.IPS_Simulator_Langevin(p)
simulator.init(pair_force_config, confinement_config)

energy_arr = []

def get_energy_callback(simulator, p, step):
    if step >= equilibration_steps:
        energy = calculate_total_energy(p.get_positions(), p.get_velocities(), epsilon=epsilon, sigma=sigma)
        energy_arr.append(energy)

sim_visualizer = SimulationVisualizer(simulator=simulator, particle_system=p, 
                                        rad=rad + 5.0, draw_interval=draw_interval, dt=0.001, show=True)
sim_visualizer.add_callback(get_energy_callback)

sim_visualizer.run_animation(num_step)